In [1]:
import pickle

countVector = pickle.load( open( "files/countVector.pickle", "rb" ) )

In [2]:
countVector

<8961258x33736 sparse matrix of type '<class 'numpy.int64'>'
	with 61759505 stored elements in Compressed Sparse Row format>

In [5]:
import pandas as pd

df = pickle.load( open( "files/df.pickle", "rb" ) )

In [6]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(countVector, df['Category'])

In [8]:
model = LatentDirichletAllocation(100)
model.fit(X_train)


AttributeError: 'LatentDirichletAllocation' object has no attribute 'transfrom'

In [9]:
XX_train = model.transform(X_train)
XX_test = model.transform(X_test)

In [11]:
pickle.dump( model, open( "models/lda_model.pickle", "wb" ) )
pickle.dump( X_test, open( "files/X_test.pickle", "wb" ) )
pickle.dump( X_train, open( "files/X_train.pickle", "wb" ) )
pickle.dump( X_test, open( "files/XX_test.pickle", "wb" ) )
pickle.dump( X_train, open( "files/XX_train.pickle", "wb" ) )

In [12]:
pickle.dump( y_train, open( "files/y_train.pickle", "wb" ) )
pickle.dump( y_test, open( "files/y_test.pickle", "wb" ) )